# 01. 对象类型（Object Types）

Python 中一切皆对象。本节把‘类型/可变性/相等与同一性/可哈希’一次讲清，避免后面写函数、类、字典时反复踩坑。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 无


## 知识点地图

- 1. 内置类型全景图：标量/序列/映射/集合
- 2. type 与 isinstance：精确类型 vs 属于某类
- 3. 可变与不可变：名字绑定 + 原地修改
- 4. == 与 is：值相等 vs 同一对象
- 5. hashable：为什么 list 不能当 dict key
- 6. 复制：浅拷贝 vs 深拷贝（避免‘以为复制了’）
- 7. 真值测试（预告）：哪些对象会被当成 False


## 自检清单（学完打勾）

- [ ] 能列出常见内置类型（int/float/bool/str/list/dict/tuple/set/None/bytes）并说出典型用途
- [ ] 会用 type()/isinstance()，知道二者区别
- [ ] 理解可变/不可变对赋值、传参、复制的影响
- [ ] 能解释 == 与 is 的区别，并知道 None 用 is None
- [ ] 理解 hashable 与 dict key / set 元素的关系


## 知识点 1：内置类型全景图：标量/序列/映射/集合

把类型按“能做什么”来记更牢：

- 标量：`int/float/bool/None`（表示一个值）
- 序列：`str/list/tuple/bytes`（有顺序、可索引/切片）
- 映射：`dict`（key -> value）
- 集合：`set/frozenset`（去重、集合运算）

学习建议：先掌握每类最常用的 2~3 个操作，再扩展细节。



In [1]:
objects = [
    42,
    3.14,
    True,
    None,
    "hello",
    [1, 2, 3],
    (1, 2),
    {1, 2, 3},
    {"a": 1, "b": 2},
    b"bytes",
]

for o in objects:
    print(f"{o!r:>12} -> {type(o)}")


          42 -> <class 'int'>
        3.14 -> <class 'float'>
        True -> <class 'bool'>
        None -> <class 'NoneType'>
     'hello' -> <class 'str'>
   [1, 2, 3] -> <class 'list'>
      (1, 2) -> <class 'tuple'>
   {1, 2, 3} -> <class 'set'>
{'a': 1, 'b': 2} -> <class 'dict'>
    b'bytes' -> <class 'bytes'>


## 知识点 2：type 与 isinstance：精确类型 vs 属于某类

- `type(x) is T`：精确类型判断（不包含子类）
- `isinstance(x, T)`：x 是否是 T 或其子类实例（更常用）

经验：业务逻辑通常用 `isinstance`，因为它兼容子类；只有在协议/序列化等场景才用 `type(x) is ...` 做严格判断。



In [2]:
class MyInt(int):
    pass

y = MyInt(7)
print(type(y) is int)        # False
print(isinstance(y, int))    # True


False
True


## 知识点 3：可变与不可变：名字绑定 + 原地修改

**不可变对象**（如 `int/str/tuple/bytes`）无法原地改变内容；你看到的“修改”其实是创建新对象并重新绑定名字。

**可变对象**（如 `list/dict/set`）可以原地修改；当多个名字引用同一对象时就会“联动”。

这是后面“可变默认参数坑”的根因（第 03 节会讲）。



In [3]:
a = "py"
print(id(a), a)
a = a + "thon"  # 新对象
print(id(a), a)

lst = [1, 2]
print(id(lst), lst)
lst.append(3)    # 原地修改
print(id(lst), lst)


1548231342384 py
1548328876592 python
1548328996224 [1, 2]
1548328996224 [1, 2, 3]


## 知识点 4：== 与 is：值相等 vs 同一对象

- `==` 比较值（通常由 `__eq__` 决定）
- `is` 比较身份（是否同一对象引用）

规则：
- 判断 `None` 用 `is None`
- 判断“内容是否相同”用 `==`



In [4]:
a = [1, 2, 3]
b = [1, 2, 3]
print(a == b)   # True
print(a is b)   # False

c = a
print(c is a)   # True


True
False
True


## 知识点 5：hashable：为什么 list 不能当 dict key

`dict` 的 key 与 `set` 的元素必须是 **可哈希（hashable）** 的：需要稳定的 `hash()` 值。

- 常见可哈希：`int/str/tuple/frozenset`（通常不可变）
- 常见不可哈希：`list/dict/set`（可变，hash 不稳定）



In [5]:
print(hash("abc"))
print(hash((1, 2)))

try:
    hash([1, 2])
except TypeError as e:
    print("list not hashable:", e)


3199865197445301356
-3550055125485641917
list not hashable: unhashable type: 'list'


## 知识点 6：复制：浅拷贝 vs 深拷贝（避免‘以为复制了’）

- 浅拷贝：复制容器本身，内部元素仍引用同一对象
- 深拷贝：递归复制内部对象

经验：优先浅拷贝；只有当内部还有可变对象且必须隔离时再深拷贝。



In [6]:
import copy

a = [[1], [2]]
b = list(a)          # 浅拷贝
c = copy.deepcopy(a) # 深拷贝

a[0].append(99)
print("a:", a)
print("b:", b)  # 受影响
print("c:", c)  # 不受影响


a: [[1, 99], [2]]
b: [[1, 99], [2]]
c: [[1], [2]]


## 知识点 7：真值测试（预告）：哪些对象会被当成 False

常见 False：
- `0/0.0`
- 空容器：`''`, `[]`, `{}`, `set()`
- `None`

你会在 if/while 中大量用到：`if not items:`。



In [7]:
values = [0, 1, "", "x", [], [1], {}, {"a": 1}, None]
for v in values:
    print(f"{v!r:>10} -> {bool(v)}")


         0 -> False
         1 -> True
        '' -> False
       'x' -> True
        [] -> False
       [1] -> True
        {} -> False
  {'a': 1} -> True
      None -> False


## 常见坑

- 不要用 is 来比较字符串/数字的值（只用于 None 或身份判断）
- 列表/字典等可变对象被多个变量引用时，原地修改会联动


## 综合小案例：实现 to_int：把输入规范化为 int

写 `to_int(x)`：
- x 是 int（但不是 bool）直接返回
- x 是 str：允许首尾空格，转换失败抛 ValueError
- 其他类型抛 TypeError



In [ ]:
def to_int(x):
    if isinstance(x, int) and not isinstance(x, bool):
        return x
    if isinstance(x, str):
        return int(x.strip())
    raise TypeError(f"cannot convert {type(x).__name__} to int")

print(to_int(10))
print(to_int(" 42 "))
try:
    to_int([1, 2])
except Exception as e:
    print(type(e).__name__, e)


## 自测题（不写代码也能回答）

- 为什么判断 None 推荐用 is None？
- 浅拷贝与深拷贝分别适合什么场景？
- 为什么 list 不能作为 dict 的 key？


## 练习题（建议写代码）

- 写 is_mutable(obj)：能识别 list/dict/set 可变、str/tuple/int 不可变。
- 写 safe_copy_matrix(m)：对二维列表做深拷贝（不使用 copy.deepcopy）。
